# **Tropos (API Feedback Generation)**
#### *This notebook handles set up to Github/Google Drive, model feedback generation, and generates docx files with feedback.*

## **--Set up**

In [1]:
#######################################################################
# Sets up Google Drive, Github, Github branch
# Installs requirements.txt and needed libraries
#######################################################################
# Mount Google Drive (optional, you'll get a prompt to authorize account)
# from google.colab import drive
# drive.mount('/content/drive')

# Start in root Colab directory to avoid nesting
%cd /content

# Clone your GitHub repo (replace with your actual repo URL)
!git clone https://github.com/ML-name/project.git
%cd project

# List all branches (optional, for checking)
# !git branch -a

# Checkout YOUR branch (!!replace "your-branch-name"!!)
!git checkout -b fix/RateLimitError origin/fix/RateLimitError

%pip install -r requirements.txt --quiet
!pip install --upgrade openai --quiet #for chatgpt
!pip install -q google-generativeai --quiet #for gemini
!pip install anthropic httpx --quiet #for claude

/content
Cloning into 'project'...
remote: Enumerating objects: 1568, done.
remote: Counting objects: 100% (346/346), done.
remote: Compressing objects: 100% (237/237), done.
remote: Total 1568 (delta 190), reused 150 (delta 108), pack-reused 1222 (from 3)
Receiving objects: 100% (1568/1568), 401.61 MiB | 16.78 MiB/s, done.
Resolving deltas: 100% (882/882), done.
Updating files: 100% (158/158), done.
/content/project
Branch 'fix/RateLimitError' set up to track remote branch 'fix/RateLimitError' from 'origin'.
Switched to a new branch 'fix/RateLimitError'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.4/46.4 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 MB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.7/322.7 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
########################################################################
# Default imports and connecting all API keys
#######################################################################
# Add src folder to python path to edit python files
import sys
sys.path.append('/content/project/')

from google.colab import userdata
import os
import anthropic
openai_key = userdata.get("conner").strip()
deepseek_key = userdata.get("deepseek").strip()
llama_key = userdata.get("groq").strip()
claude_key = userdata.get("claude").strip()


os.environ["OPENAI_API_KEY"] = openai_key

from openai import OpenAI
client = OpenAI(api_key=deepseek_key, base_url="https://api.deepseek.com")

os.environ["LLAMA_API_URL"] = "https://api.groq.com/openai/v1"  # For Ollama
os.environ["LLAMA_API_KEY"] = llama_key  #
os.environ["CLAUDE_API_KEY"] = claude_key

import google.generativeai as genai
genai.configure(api_key=userdata.get("gemini").strip())


## **--Feedback Generation Prep**

In [14]:
#######################################################################
# Ensure past generated output is cleared
# (comment out if do not want)
#######################################################################
from utils.file_utils import clear_directory_if_exists
output_dir = "/content/project/data/generated_output"
clear_directory_if_exists(output_dir)

✅ The generated_output directory does not exist. No worries!


In [13]:
#######################################################################
# Set up paths for feedback engine
#######################################################################
from tropos import test_feedback_console
reqs_file = "./data/raw/Requirements.docx"
examples_dir = "./data/raw/Student_Submissions"
targets_dir = "./data/unmarked_raw"
outputs_dir = "./data/generated_output"
profile_save_path = "./data/generated_profile/instructor_profile.txt"

In [12]:
# --------------------------------------
# Step 1: Generate Instructor Profile
# --------------------------------------

from tropos.models.prompt_builder import generate_full_instructor_profile, load_profile_from_txt

# Paths
examples_dir = "./data/raw/Student_Submissions"
reqs_file = "./data/raw/Requirements.docx"
profile_save_path = "./data/generated_profile/instructor_profile.txt"

batch_size = 5
model_name = "gpt-4o"

# Generate the Instructor Profile
generate_new_profile = True

if generate_new_profile:
    final_profile = generate_full_instructor_profile(
        examples_dir=examples_dir,
        requirements_path=reqs_file,
        batch_size=batch_size,
        model_name=model_name,
        debug=True
    )

    os.makedirs(os.path.dirname(profile_save_path), exist_ok=True)
    with open(profile_save_path, "w", encoding="utf-8") as f:
        f.write(final_profile)

    print(f"📂 Final Instructor Profile saved to {profile_save_path}")
else:
    final_profile = load_profile_from_txt(profile_save_path)
    print("✅ Loaded Instructor Profile successfully!")


ValueError: ❌ No examples found in directory.

In [7]:
from tropos import test_feedback_console
from tropos.models.prompt_builder import load_profile_from_txt

profile_save_path = "./data/generated_profile/instructor_profile.txt"
final_profile = load_profile_from_txt(profile_save_path)

test_feedback_console(
    prompt_type="ProfileShot",
    model="gpt-4o",
    requirements_path="./data/raw/Requirements.docx",
    example_dir="./data/raw/Student_Submissions",
    target_dir="./data/unmarked_raw",
    output_dir="./data/generated_output",
    output_mode="pretty",
    max_examples=None,
    profile_text=final_profile  # <-- NOW PASSING PROFILE TEXT!
)

FileNotFoundError: ❌ Profile file not found: ./data/generated_profile/instructor_profile.txt

## **--Currently Running Models**

In [ ]:
# # GPT-4o
# test_feedback_console(
#     prompt_type="FewShot",
#     model="gpt-4o",
#     requirements_path=reqs,
#     example_dir=examples,
#     target_dir=targets,
#     output_dir=outputs,
#     output_mode="pretty",
#     max_examples=3
# )


## **-- Different Models** (Commented out for now)

In [ ]:
# # GPT-4.1
# test_feedback_console(
#     prompt_type="FewShot",
#     model="gpt-4.1",
#     requirements_path=reqs,
#     example_dir=examples,
#     target_dir=targets,
#     output_dir=outputs,
#     output_mode="raw",
#     max_examples=3
# )

In [ ]:
# # GPT-4.1
# test_feedback_console(
#     prompt_type="FewShot",
#     model="gpt-4.1",
#     requirements_path=reqs,
#     example_dir=examples,
#     target_dir=targets,
#     output_dir=outputs,
#     output_mode="pretty",
#     max_examples=3
# )

In [ ]:
# # DeepSeek
# test_feedback_console(
#     prompt_type="FewShot",
#     model="deepseek-chat",
#     requirements_path=reqs,
#     example_dir=examples,
#     target_dir=targets,
#     output_dir=outputs,
#     output_mode="pretty",
#     max_examples=3
# )

In [ ]:
# # Gemini
# test_feedback_console(
#     prompt_type="FewShot",
#     model="gemini-1.5-pro-latest",
#     requirements_path=reqs,
#     example_dir=examples,
#     target_dir=targets,
#     output_dir=outputs,
#     output_mode="pretty",
#     max_examples=3
# )

In [ ]:
# # Llama (different prompt_type)
# test_feedback_console(
#     prompt_type="FewShot-Llama",
#     model="meta-llama/llama-4-scout-17b-16e-instruct",
#     requirements_path=reqs,
#     example_dir=examples,
#     target_dir=targets,
#     output_dir=outputs,
#     output_mode="pretty",
#     max_examples=3
# )

In [ ]:
# # Claude
# test_feedback_console(
#     prompt_type="FewShot",
#     model="claude-3-opus-20240229",
#     requirements_path=reqs,
#     example_dir=examples,
#     target_dir=targets,
#     output_dir=outputs,
#     output_mode="pretty",
#     max_examples=3
# )